In [1]:
import numpy as np
import pandas as pd

from findlay2025a import core
from findlay2025a.constants import Experiments as Exps
from findlay2025a.constants import Files

In [2]:
experiment_display_names = {
    Exps.NOD: "Novelty",
    Exps.COW: "Locomotion",
    Exps.CTN: "Dual",
}

exclusions = [
    ("CNPIX5-Alessandro", Exps.NOD, ("ext_wake_incline",)),
    ("CNPIX6-Eugene", Exps.NOD, ("rebound", "bsl_decline", "ext_wake_incline")),
    ("CNPIX10-Charles", Exps.NOD, ("ext_wake_incline",)),
    ("CNPIX11-Adrian", Exps.NOD, ("ext_wake_incline",)),
    ("CNPIX11-Adrian", Exps.COW, ("ext_wake_incline",)),
    ("CNPIX17-Hans", Exps.COW, ("ext_wake_incline",)),
]

contrast_display_names = {
    "cx_mean_zlog_delta_nrem_rebound": "Cortical SWA Rebound",
    "cx_mean_zlog_delta_nrem_rec_decline": "Cortical SWA Decline",
    "spw_rate_rel2total_nrem_rebound": "SPW Rate Rebound",
    "spw_rate_rel2total_nrem_rec_decline": "SPW Rate Decline",
    "spw_mean_zlog_amp_nrem_rebound": "SPW Amplitude Rebound",
    "spw_mean_zlog_amp_nrem_rec_decline": "SPW Amplitude Decline",
    "ripple_rate_rel2total_nrem_rebound": "Ripple Rate Rebound",
    "ripple_rate_rel2total_nrem_rec_decline": "Ripple Rate Decline",
    "ripple_mean_zfreq_nrem_rebound": "Ripple Frequency Rebound",
    "ripple_mean_zfreq_nrem_rec_decline": "Ripple Frequency Decline",
    "dspk_mean_zlog_height_nrem_rebound": "DSPK Amplitude Rebound",
    "dspk_mean_zlog_height_nrem_rec_decline": "DSPK Amplitude Decline",
}

In [3]:
def apply_exclusions(df: pd.DataFrame) -> pd.DataFrame:
    essential_cols = ["subject", "experiment"]
    assert all(col in df.columns for col in essential_cols), (
        f"Missing one or more essential columns: {essential_cols}"
    )

    for sub, exp, conds in exclusions:
        cols = [
            col for col in df.columns if any(cond in col for cond in conds)
        ]  # Columns whose values to drop
        rows = (df["subject"] == sub) & (df["experiment"] == exp)
        df.loc[rows, cols] = np.NaN

    return df


def use_display_names(df: pd.DataFrame, minimize: bool = True) -> pd.DataFrame:
    essential_cols = ["subject", "experiment"]
    assert all(col in df.columns for col in essential_cols), (
        f"Missing one or more essential columns: {essential_cols}"
    )

    if minimize:
        df = df.loc[:, essential_cols + list(contrast_display_names)]

    df["experiment"] = df["experiment"].map(experiment_display_names)
    df.rename(columns=contrast_display_names, inplace=True)

    return df


def get_essential_contrasts() -> pd.DataFrame:
    nb = core.get_project("seahorse")
    df = pd.read_parquet(
        nb.get_project_file(Files.COMBINED_CONDITION_CONTRASTS)
    ).reset_index()
    return use_display_names(apply_exclusions(df))

In [4]:
nb = core.get_project("seahorse")
df = get_essential_contrasts()
df.to_parquet(nb.get_project_file("condition_contrasts.pqt"))